In [ ]:
import pandas as pd
import csv
from bs4 import BeautifulSoup

In [ ]:
# You may need this if beautfulsoup is not installed
# !pip install beautifulsoup4

In [ ]:
input_svg_filepath = './data/counties.svg' # USA_Counties_with_FIPS_and_names
output_svg_filepath = './data/Unemployment_Map.svg'
unemployment_datapath = './data/unemployment09.csv'

In [ ]:
# Read in unemployment rates
unemployment = {}
reader = csv.reader(open(unemployment_datapath), delimiter=",")
for row in reader:
    try:
        full_fips = row[1] + row[2]
        rate = float( row[8].strip() )
        unemployment[full_fips] = rate
    except:
        pass

In [ ]:
# Load the SVG map
svg = open(input_svg_filepath, 'r').read()

In [ ]:
# Load into Beautiful Soup
soup = BeautifulSoup(svg, selfClosingTags=['defs','sodipodi:namedview'])

In [ ]:
# Find counties
paths = soup.findAll('path')

In [ ]:
# Map colors
colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]

In [ ]:
# County style
path_style = 'font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;stroke-width:0.1;stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;marker-start:none;stroke-linejoin:bevel;fill:'

In [ ]:
df = pd.read_csv(unemployment_datapath, header=None, dtype=str)
df.loc[:, 'FIPS'] = 'FIPS_' + df.loc[:, 1] + df.loc[:, 2]
df

In [ ]:
unemployment_mapping = dict(zip(df.loc[:, 'FIPS'], df.loc[:, 8].astype(float)))

In [ ]:
# Color the counties based on unemployment rate
for p in paths:     
    if p['id'] not in ["State_Lines", "separator"]:
        # pass
        try:
            rate = unemployment_mapping[p['id']]
        except KeyError as err:
            pass             
        if rate > 10:
            color_class = 5
        elif rate > 8:
            color_class = 4
        elif rate > 6:
            color_class = 3
        elif rate > 4:
            color_class = 2
        elif rate > 2:
            color_class = 1
        else:
            color_class = 0
        color = colors[color_class]
        p['style'] = path_style + color

In [ ]:
# Output map
f = open(output_svg_filepath, "w")
f.write(soup.prettify())
f.close()
# print(soup.prettify())